In [2]:
import numpy as np
from lmfit import Model, fit_report, minimize
import csv

print('Matlab ref omega: ', 1.752690409810814)
method = 'basinhopping'
# Define the function to be fitted
def func(x, a, b, omega, phi, c):
    return a * np.exp(-b * x) * np.sin(omega * x + phi) + c

# Define the residual function
def residual(params, x, y):
    return y - func(x, **params)

# Load data from text files
# xdata = np.loadtxt('xdata_mask20.txt')
# ydata = np.loadtxt('ydata_mask20.txt')
xdata = np.loadtxt('xdata.txt')
ydata = np.loadtxt('ydata_noisy5dB.txt')



# # Define the boolean mask
# mask = np.logical_not(np.isnan(ydata))
# # Apply the mask to the data
# xdata = xdata[mask]
# ydata = ydata[mask]

# Define the model to be fitted
model = Model(func)


# Set initial parameter values and bounds
params = model.make_params(a=1.0, b=1.0, omega=1.0, phi=1.0, c=1.0)
params['a'].min = 0.0
params['b'].min = 0.0
params['omega'].min = 0.0
params['phi'].min = -np.pi
params['phi'].max = np.pi
params['a'].max = 5.0
params['b'].max = 5.0
params['omega'].max = 5.0
params['c'].min=0.0
params['c'].max=5.0

# Define the number of times to run the fit
n_runs = 12

# Create a list to store the results
results = []

# Run the fit n_runs times and record the results
for i in range(n_runs):
    # Mask 20% of the data
#     n_mask = int(len(ydata) * 0.2)
#     mask_indices = np.random.choice(len(ydata), size=n_mask, replace=False)
#     ydata_masked = ydata.copy()
#     ydata_masked[mask_indices] = np.nan
    
#     mask = np.logical_not(np.isnan(ydata_masked))
#     # Apply the mask to the data
#     xdata = xdata[mask]
#     ydata = ydata[mask]
    
    # Fit the model to the data using the least squares method
    result = minimize(residual, params, args=(xdata, ydata), method= method)
    print(result.params['b'].value, result.params['omega'].value, result.chisqr)
    # Record the results
    results.append({
        'run': i + 1,
        'method': method,
        'b': result.params['b'].value,
        'omega': result.params['omega'].value,
        'residual': result.chisqr
    })

# Write the results to a CSV file
with open('./data/results_{0}_mask20.csv'.format(method), 'w', newline='') as csvfile:
    fieldnames = ['run', 'method', 'b', 'omega', 'residual']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for result in results:
        writer.writerow(result)

# Print the fit results for the last run
print("Results using least sq:")
# print(fit_report(result))


Matlab ref omega:  1.752690409810814
0.18683616024354255 1.7625179337658436 29.92526282698033
0.18683641812537177 1.7625177989297192 29.925262826979637
0.18683618843218702 1.7625177395141622 29.925262826980482
0.18683631495369696 1.7625179183461381 29.92526282698035


KeyboardInterrupt: 